In [40]:
import pandas as pd

In [41]:
cleaned_league = pd.read_csv("complete_cleaned.csv")
counter_df = pd.read_csv("counters.csv")

In [42]:
cleaned_league = cleaned_league.drop(columns=['Unnamed: 0.1', "Unnamed: 0"])

In [43]:
counter_df.head()

,Champion,counters champ,champ counters
0,nilah,"('Smolder', 45)","('Twitch', 61)"
1,nilah,"('Xayah', 46)","('Samira', 59)"
2,nilah,"('Draven', 49)","('Varus', 58)"
3,nilah,"('Vayne', 50)","(""Kai'Sa"", 56)"
4,nilah,"('Jinx', 51)","('Miss Fortune', 56)"


In [46]:
def format_champion(champ):
        return champ.lower().replace(" ", "-").replace("'", "").replace("nunu-&-willump", "nunu-willump").replace("dr.-mundo", "dr-mundo")

def get_picks(df): 
    blue = df[df['side'] == 'Blue'][['pick1', 'pick2', 'pick3', 'pick4', 'pick5']].values.flatten().tolist()
    red = df[df['side'] == 'Red'][['pick1', 'pick2', 'pick3', 'pick4', 'pick5']].values.flatten().tolist()

    blue = [format_champion(champ) for champ in blue]
    red = [format_champion(champ) for champ in red]

    num_counters = count_counters([red, blue])
    counter_data = {"Red": num_counters[0], "Blue": num_counters[1]}
    df["Number of Counter Picks"] = df["side"].map(counter_data)
    return df

def count_counters(team): 
    counter_df = pd.read_csv("counters.csv")

    # Standardizing the champion names in counter_df
    counter_df['Champion'] = counter_df['Champion'].str.lower().str.replace(" ", "-").str.replace("'", "")
    counter_df['counters champ'] = counter_df['counters champ'].apply(lambda x: eval(x) if isinstance(x, str) else x)

    red = team[0]
    blue = team[1]

    countered_red_champs = set()
    countered_blue_champs = set()

    # Count blue team's counter picks (against red team)
    for red_champ in red:
        champ_counters = {format_champion(champ[0]) for champ in counter_df[counter_df['Champion'] == red_champ]["counters champ"].dropna().tolist()}
        for blue_champ in blue:
            if blue_champ in champ_counters:
                countered_red_champs.add(red_champ)
                break  # Only count one counter per champion

    num_blue_counters = len(countered_red_champs)

    # Count red team's counter picks (against blue team)
    for blue_champ in blue:
        champ_counters = {format_champion(champ[0]) for champ in counter_df[counter_df['Champion'] == blue_champ]["counters champ"].dropna().tolist()}
        for red_champ in red:
            if red_champ in champ_counters:
                countered_blue_champs.add(blue_champ)
                break  # Only count one counter per champion

    num_red_counters = len(countered_blue_champs)

    return [num_red_counters, num_blue_counters]



In [47]:

updated_league = cleaned_league.groupby("gameid").apply(get_picks)
updated_league


C:\Users\Viki Shi\AppData\Local\Temp\ipykernel_17108\25167177.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  updated_league = cleaned_league.groupby("gameid").apply(get_picks)


gameid  side    pick1      pick2  \
gameid                                                                    
1000-1175          3632             1000-1175  Blue     Ekko    Hecarim   
                   3633             1000-1175   Red     Gnar     Gragas   
10000-10000_game_1 122608  10000-10000_game_1  Blue      Jax    Sejuani   
                   122609  10000-10000_game_1   Red  K'Sante         Vi   
10000-10000_game_2 122610  10000-10000_game_2  Blue      Jax     Maokai   
...                                       ...   ...      ...        ...   
WMC2TMNT1/320038   23459     WMC2TMNT1/320038   Red    Urgot    Sejuani   
WMC2TMNT1/320040   23460     WMC2TMNT1/320040  Blue    Jayce  Jarvan IV   
                   23461     WMC2TMNT1/320040   Red   Maokai    Sejuani   
WMC2TMNT1/320050   23462     WMC2TMNT1/320050  Blue    Fiora    Sejuani   
                   23463     WMC2TMNT1/320050   Red     Shen     Gragas   

                              pick3   pick4     pick5     ban1     ban2  \
gameid                                                                    
1000-1175          3632    Malzahar    Ashe     Karma  LeBlanc  Trundle   
                   3633      Viktor   Sivir     Braum    Janna   Maokai   
10000-10000_game_1 122608     Akali  Kai'Sa  Nautilus      NaN      NaN   
                   122609     Galio    Zeri   Alistar      NaN      NaN   
10000-10000_game_2 122610      Ryze   Xayah     Rakan      NaN      NaN   
...                             ...     ...       ...      ...      ...   
WMC2TMNT1/320038   23459    Orianna   Xayah     Rakan    Taric  Kalista   
WMC2TMNT1/320040   23460   Malzahar   Xayah     Taric  Camille   Ezreal   
                   23461    Orianna  Twitch     Rakan     Ornn  Kalista   
WMC2TMNT1/320050   23462     Viktor   Xayah     Braum  Caitlyn    Jayce   
                   23463       Azir   Varus     Rakan  Camille  Kalista   

                               ban3      ban4       ban5  result  \
gameid                                                             
1000-1175          3632        Shen       NaN        NaN       0   
                   3633    Vladimir       NaN        NaN       1   
10000-10000_game_1 122608       NaN       NaN        NaN       0   
                   122609       NaN       NaN        NaN       1   
10000-10000_game_2 122610       NaN       NaN        NaN       1   
...                             ...       ...        ...     ...   
WMC2TMNT1/320038   23459       Ornn  Malzahar      Karma       1   
WMC2TMNT1/320040   23460   Tristana   Nidalee    Rek'Sai       1   
                   23461    Caitlyn    Syndra       Azir       0   
WMC2TMNT1/320050   23462   Tristana    Twitch  Jarvan IV       0   
                   23463       Ornn   Orianna     Syndra       1   

                           Number of Counter Picks  
gameid                                              
1000-1175          3632                          1  
                   3633                          0  
10000-10000_game_1 122608                        0  
                   122609                        1  
10000-10000_game_2 122610                        1  
...                                            ...  
WMC2TMNT1/320038   23459                         3  
WMC2TMNT1/320040   23460                         0  
                   23461                         1  
WMC2TMNT1/320050   23462                         1  
                   23463                         1  

[167140 rows x 14 columns]

In [50]:
updated_league[updated_league["Number of Counter Picks"] != 0].shape[0]

80565